In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

In [4]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 13s 6ms/step - loss: 0.2176 - accuracy: 0.9361 - val_loss: 0.0931 - val_accuracy: 0.9708
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0977 - accuracy: 0.9707 - val_loss: 0.0794 - val_accuracy: 0.9745
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0676 - accuracy: 0.9788 - val_loss: 0.0661 - val_accuracy: 0.9798
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0544 - accuracy: 0.9826 - val_loss: 0.0690 - val_accuracy: 0.9797
Epoch 5/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0431 - accuracy: 0.9859 - val_loss: 0.0594 - val_accuracy: 0.9822


In [5]:
%tensorboard --logdir logs/fit

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [8]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [9]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [10]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [11]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24330781400203705, Accuracy: 92.89666748046875, Test Loss: 0.11704090237617493, Test Accuracy: 96.37000274658203
Epoch 2, Loss: 0.1037142351269722, Accuracy: 96.91999816894531, Test Loss: 0.08793012797832489, Test Accuracy: 97.25999450683594
Epoch 3, Loss: 0.07229021936655045, Accuracy: 97.78166198730469, Test Loss: 0.07134979218244553, Test Accuracy: 97.58999633789062
Epoch 4, Loss: 0.05438503250479698, Accuracy: 98.28333282470703, Test Loss: 0.06631053239107132, Test Accuracy: 97.80999755859375
Epoch 5, Loss: 0.04283410683274269, Accuracy: 98.6433334350586, Test Loss: 0.06599347293376923, Test Accuracy: 98.02999877929688


In [12]:
%tensorboard --logdir logs/gradient_tape